Creating summary snapshots of the dataset

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt
import glob
import scanpy as sc
import json
import os

import geopandas as gpd
import json
from shapely.geometry import Polygon, Point
import glob
import pandas as pd
import pathlib

import cv2
import matplotlib.pyplot as plt

Add paths to experiment and segmentation

In [ ]:
experiment = "SI_d8pi"

segmentation_path = (
    f"visium_hd/segmentation/segmentation_outputs/{experiment}_segmentation.png"
)

Read in the segmentation and high res image

In [ ]:
import PIL

PIL.Image.MAX_IMAGE_PIXELS = 2382717010
segmentation = imageio.imread(segmentation_path)

In [ ]:
visium_path = "VisiumHD_data/LJI_001_visiumhd_SI"
experiment_spatial_path = os.path.join(
    visium_path, "count_outputs", f"visium_hd_count_{experiment}", "outs", "spatial"
)
highres_image = glob.glob(os.path.join(experiment_spatial_path, "*used.tif"))[0]

In [ ]:
def read_dapi_image(path: str) -> np.ndarray:
    img_fpath = pathlib.Path(path)
    img = imageio.imread(img_fpath)
    print(img.shape)
    return img


maxed_visium = read_dapi_image(highres_image)

Reading in all the capture spot positions and their expression

In [ ]:
dir_base = f"VisiumHD_data/LJI_001_visiumhd_SI/count_outputs/visium_hd_count_{experiment}/outs/binned_outputs/square_002um/"
# Load Visium HD data
raw_h5_file = dir_base + "filtered_feature_bc_matrix.h5"
adata = sc.read_10x_h5(raw_h5_file)

# Load the Spatial Coordinates
tissue_position_file = dir_base + "spatial/tissue_positions.parquet"
df_tissue_positions = pd.read_parquet(tissue_position_file)

# Set the index of the dataframe to the barcodes
df_tissue_positions = df_tissue_positions.set_index("barcode")

# Create an index in the dataframe to check joins
df_tissue_positions["index"] = df_tissue_positions.index

# Adding the tissue positions to the meta data
adata.obs = pd.merge(adata.obs, df_tissue_positions, left_index=True, right_index=True)

# Create a GeoDataFrame from the DataFrame of coordinates
geometry = [
    Point(xy)
    for xy in zip(
        df_tissue_positions["pxl_col_in_fullres"],
        df_tissue_positions["pxl_row_in_fullres"],
    )
]
gdf_coordinates = gpd.GeoDataFrame(df_tissue_positions, geometry=geometry)

Finding the edges in the segmentation mask. Used sin and cos just in case one of them is 0 for a paritcular point

In [ ]:
if segmentation.dtype != np.uint8:
    segmentation_uint8_sin = (
        255
        * np.sin(segmentation)
        * (segmentation - np.min(segmentation))
        / (np.max(segmentation) - np.min(segmentation))
    ).astype(np.uint8)
    segmentation_uint8_cos = (
        255
        * np.cos(segmentation)
        * (segmentation - np.min(segmentation))
        / (np.max(segmentation) - np.min(segmentation))
    ).astype(np.uint8)
edges_sin = cv2.Canny(segmentation_uint8_sin, threshold1=0, threshold2=0)

edges_sin[edges_sin > 0] = 255

edges_cos = cv2.Canny(segmentation_uint8_cos, threshold1=0, threshold2=0)
edges_cos[edges_cos > 0] = 255

edges = np.clip(edges_sin + edges_cos, 0, 255)

Midrange moneyshot

In [ ]:
overlay = np.zeros_like(maxed_visium)

pink_color = (255, 255, 0)

# Set the pink color where the edges are (where edges == 255)
overlay[edges >= 254] = pink_color

# Combine the original image with the pink overlay using alpha blending
alpha = 1  # Transparency factor
result = cv2.addWeighted(maxed_visium, alpha - 0.4, overlay, alpha, 2)

In [ ]:
from matplotlib.patches import Rectangle

window_low_x = 9000
window_low_y = 14700
window_high_x = 12000
window_high_y = 17000

min_y = 500
max_y = 800
min_x = 500
max_x = 800

fig, ax0 = plt.subplots(1, 1, figsize=(10, 10))
rectangle = Rectangle(
    (min_y, min_x),
    max_y - min_y,
    max_x - min_x,
    linewidth=4,
    edgecolor="black",
    facecolor="none",
)
ax0.imshow(result[window_low_x:window_high_x, window_low_y:window_high_y])
ax0.add_patch(rectangle)
fig.savefig("mid_zoom.pdf")
plt.show()

Coloring cells by gene expression

In [ ]:
window_low_x = 9000
window_low_y = 14700
window_high_x = 12000
window_high_y = 17000

In [ ]:
temp_seg = segmentation.copy()[window_low_x:window_high_x, window_low_y:window_high_y]

In [ ]:
corrected_x = adata.obs["pxl_row_in_fullres"] - window_low_x
corrected_y = adata.obs["pxl_col_in_fullres"] - window_low_y

ids_corrected = np.where(
    (corrected_x > 0)
    & (corrected_x < (window_high_x - window_low_x))
    & (corrected_y > 0)
    & (corrected_y < (window_high_y - window_low_y))
)[0]

In [ ]:
gene = "Reg3b"

In [ ]:
gene_counts = adata.X[:, adata.var.index == gene].toarray().flatten()[ids_corrected]

In [ ]:
corrected_row = adata.obs["pxl_col_in_fullres"].values[ids_corrected] - window_low_y
corrected_col = adata.obs["pxl_row_in_fullres"].values[ids_corrected] - window_low_x

sub_segmentation = temp_seg.copy()

dic = {}
for i in range(len(ids_corrected)):
    value = sub_segmentation[int(corrected_col[i]), int(corrected_row[i])]
    if value not in dic:
        dic[value] = 0
    dic[value] += gene_counts[i]
del dic[0]

dic_values = np.array(list(dic.values()))

for k in list(dic.keys()):
    sub_segmentation[sub_segmentation == k] = int(dic.get(k) + 1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, ListedColormap
from matplotlib.cm import viridis, magma, Blues, plasma

# Assuming sub_segmentation is your 2D grayscale image array
# Set vmax to 3 as specified, and normalize pixel values
vmax = 3
norm = Normalize(vmin=0.1, vmax=vmax)

# Create a mask for non-zero pixels
mask = sub_segmentation != 0

# Apply the colormap to the entire array at once
colored_image = np.zeros((*sub_segmentation.shape, 4))
colored_image[mask] = Blues(norm(sub_segmentation[mask]))

# Set background color to white for zero pixels
colored_image[~mask] = [1, 1, 1, 1]  # RGBA for white

# Create a custom colormap that includes white for zero values
colors = np.vstack(([1, 1, 1, 1], Blues(np.linspace(0, 1, 255))))
custom_cmap = ListedColormap(colors)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 8), dpi=300)
im = ax.imshow(sub_segmentation, cmap=custom_cmap, norm=norm)
ax.axis("off")  # Hide axes for better visualization

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label(f"{gene} expression")

plt.title(f"{gene} expression per cell")
plt.tight_layout()
plt.savefig("mid_zoom_reg3b.pdf")
plt.show()
plt.close()

Very zoom in

In [ ]:
overlay = np.zeros_like(maxed_visium)

pink_color = (255, 0, 0)

# Set the pink color where the edges are (where edges == 255)
overlay[edges >= 254] = pink_color

# Combine the original image with the pink overlay using alpha blending
alpha = 1  # Transparency factor
result = cv2.addWeighted(maxed_visium, alpha - 0.4, overlay, alpha, 2)

In [ ]:
gene = None

In [ ]:
import matplotlib.patches as patches
from matplotlib import cm
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

fig, ax = plt.subplots()
indices = np.where(
    (adata.obs["pxl_col_in_fullres"].values > (window_low_y + min_y))
    & (adata.obs["pxl_col_in_fullres"].values < (window_low_y + max_y))
    & (adata.obs["pxl_row_in_fullres"].values > (window_low_x + min_x))
    & (adata.obs["pxl_row_in_fullres"].values < (max_x + window_low_x))
)[0]
ax.imshow(
    result[
        window_low_x + min_x : window_low_x + max_x,
        window_low_y + min_y : window_low_y + max_y,
    ]
)
col_scatter = np.array(list(adata.obs["pxl_col_in_fullres"].values[indices])) - (
    window_low_y + min_y
)
row_scatter = np.array(list(adata.obs["pxl_row_in_fullres"].values[indices])) - (
    window_low_x + min_x
)

square_side = 16  # Desired side length of the squares
transparency = 0.1  # Transparency level for the fill (0 = transparent, 1 = opaque)

# Calculate the area of the squares to use in the s parameter
area = [square_side**2 for i in range(len(indices))]

# Get the viridis colormap
viridis = cm.get_cmap("viridis")


# Plotting squares with transparent fill and opaque edges
if gene == None:
    colors = np.log(np.sum(adata.X[indices, :].toarray(), axis=1))
    # Normalize the colors to the range [0, 1]
    norm_colors = colors / np.max(colors)

    # Apply the colormap to the normalized values
    edge_colors = viridis(norm_colors)

    # greater_zero = np.where(np.array(colors) == 'r')[0]
    transparency = 0.3

    for x, y, s, c in zip(col_scatter, row_scatter, area, edge_colors):
        size = (
            s**0.5
        ) / 2  # Convert area to half the length of a side for the square marker
        rect = patches.Rectangle(
            (x - size, y - size),
            2 * size,
            2 * size,
            linewidth=0.8,
            edgecolor=c,
            facecolor=(0, 0, 0, 0),
        )
        ax.add_patch(rect)
    # Create a ScalarMappable for the colorbar
    norm = Normalize(vmin=np.min(colors), vmax=np.max(colors))
    sm = ScalarMappable(norm=norm, cmap=viridis)
    sm.set_array(
        []
    )  # You can use sm.set_array(norm_colors) if you want a specific range

    # Add the colorbar
    plt.colorbar(sm, ax=ax)
else:
    colors = np.log(np.array(adata.X[indices, adata.var.index == gene]).flatten())
    # Normalize the colors to the range [0, 1]
    norm_colors = colors / np.max(colors)

    # Apply the colormap to the normalized values
    edge_colors = viridis(norm_colors)

    # greater_zero = np.where(np.array(colors) == 'r')[0]
    transparency = 0.1

    for x, y, s, c in zip(col_scatter, row_scatter, area, edge_colors):
        size = (
            s**0.5
        ) / 2  # Convert area to half the length of a side for the square marker
        rect = patches.Rectangle(
            (x - size, y - size),
            2 * size,
            2 * size,
            linewidth=0.3,
            edgecolor=c,
            facecolor=(0, 0, 0, 0),
        )
        ax.add_patch(rect)
    # Create a ScalarMappable for the colorbar
    norm = Normalize(vmin=np.min(colors), vmax=np.max(colors))
    sm = ScalarMappable(norm=norm, cmap=viridis)
    sm.set_array(
        []
    )  # You can use sm.set_array(norm_colors) if you want a specific range

    # Add the colorbar
    plt.colorbar(sm, ax=ax)
# plt.scatter(col_scatter, row_scatter, s=area, alpha=1, c='r', marker='s', linewidth=0, )
plt.title("Transcript abundances per capture spot")
plt.savefig("close_zoom.pdf")
plt.show()

Far shot

In [ ]:
from matplotlib.patches import Rectangle

window_low_x = 9000
window_low_y = 14700
window_high_x = 12000
window_high_y = 17000

fig, ax0 = plt.subplots(1, 1, figsize=(10, 10))
rectangle = Rectangle(
    (window_low_y, window_low_x),
    window_high_y - window_low_y,
    window_high_x - window_low_x,
    linewidth=2,
    edgecolor="black",
    facecolor="none",
)
ax0.imshow(result)
ax0.add_patch(rectangle)
fig.savefig("full_room.pdf")
plt.show()